In [70]:
import pandas as pd
from backend.data_center.kline_data.kline_data_collector import *
tv = KlineDataCollector()
file_abspath = tv.get_abspath(symbol='BTC', interval=Interval.in_daily)
df = pd.read_csv(f"{file_abspath}")
df

,datetime,symbol,open,high,low,close,volume
0,2017-08-17 08:00:00,Binance:BTCUSDT,4261.48,4485.39,4200.74,4285.08,795.150377
1,2017-08-18 08:00:00,Binance:BTCUSDT,4285.08,4371.52,3938.77,4108.37,1199.888264
2,2017-08-19 08:00:00,Binance:BTCUSDT,4108.37,4184.69,3850.00,4139.98,381.309763
3,2017-08-20 08:00:00,Binance:BTCUSDT,4120.98,4211.08,4032.62,4086.29,467.083022
4,2017-08-21 08:00:00,Binance:BTCUSDT,4069.13,4119.62,3911.79,4016.00,691.743060
...,...,...,...,...,...,...,...
2603,2024-10-02 08:00:00,Binance:BTCUSDT,60804.92,62390.31,60000.00,60649.28,31455.756900
2604,2024-10-03 08:00:00,Binance:BTCUSDT,60649.27,61477.19,59828.11,60752.71,26411.895000
2605,2024-10-04 08:00:00,Binance:BTCUSDT,60752.72,62484.85,60459.90,62086.00,21299.146910
2606,2024-10-05 08:00:00,Binance:BTCUSDT,62086.00,62370.56,61689.26,62058.00,7848.591930


In [63]:
df['prev_low_1'] = df['low'].shift(1)
df['prev_high_1'] = df['high'].shift(1)
df['next_close_3'] = df['close'].shift(-3)
df['next_close_2'] = df['close'].shift(-2)
df['next_close_1'] = df['close'].shift(-1)
df['next_close_5'] = df['close'].shift(-5)
df['next_open_2'] = df['open'].shift(-2)
df['next_open_1'] = df['open'].shift(-1)
df.dropna(inplace=True)
# calculate the close price in a quarter at the end of the day
df['attack_buy_pattern'] = df['close'] < df['prev_low_1'] 
df['prev_is_abp'] = df['attack_buy_pattern'].shift(1)
df.dropna(inplace=True)


In [67]:
df['buy_sig'] = (df['prev_is_abp'] == True) & (df['close']>df['prev_high_1'])
df[df['buy_sig'] == True]


,datetime,symbol,open,high,low,close,volume,prev_low_1,prev_high_1,next_close_3,next_close_2,next_close_1,next_close_5,attack_buy_pattern,prev_is_abp,buy_sig,next_open_2,next_open_1,take_profit
179,2018-02-12 08:00:00,Binance:BTCUSDT,8063.82,8989.00,8053.00,8903.00,41987.850490,7726.53,8549.00,10000.09,9449.99,8539.90,11039.55,False,True,True,8535.17,8903.00,False
285,2018-05-29 08:00:00,Binance:BTCUSDT,7099.00,7540.00,7032.95,7461.29,39406.839520,7058.02,7437.00,7521.01,7485.01,7375.96,7714.26,False,True,True,7382.07,7467.98,False
301,2018-06-14 08:00:00,Binance:BTCUSDT,6292.78,6720.00,6260.00,6635.98,44478.130837,6118.68,6627.60,6449.61,6483.98,6388.90,6741.21,False,True,True,6380.01,6638.67,False
316,2018-06-29 08:00:00,Binance:BTCUSDT,5851.01,6300.00,5780.00,6197.92,33911.287278,5827.00,6173.01,6615.29,6356.81,6390.07,6586.98,False,True,True,6391.08,6197.92,True
444,2018-11-04 08:00:00,Binance:BTCUSDT,6388.00,6525.00,6359.00,6485.85,10592.394943,6345.00,6439.97,6578.46,6519.11,6468.99,6419.99,False,True,True,6468.99,6485.85,False
498,2018-12-28 08:00:00,Binance:BTCUSDT,3567.89,3887.25,3540.04,3839.26,45964.304987,3535.00,3813.98,3702.90,3801.91,3695.32,3858.56,False,True,True,3696.71,3839.00,False
515,2019-01-14 08:00:00,Binance:BTCUSDT,3477.56,3671.87,3467.02,3626.09,35235.211215,3441.30,3611.10,3616.21,3591.84,3553.06,3665.30,False,True,True,3553.06,3626.08,False
565,2019-03-05 08:00:00,Binance:BTCUSDT,3716.10,3877.10,3703.55,3857.73,32962.536162,3670.69,3830.00,3864.89,3873.64,3861.84,3916.82,False,True,True,3861.84,3857.58,True
662,2019-06-10 08:00:00,Binance:BTCUSDT,7627.57,8020.00,7511.00,7982.75,36756.078468,7506.66,7935.00,8218.54,8127.64,7884.90,8808.70,False,True,True,7884.90,7981.00,False
781,2019-10-07 08:00:00,Binance:BTCUSDT,7855.30,8299.92,7762.00,8190.09,52202.072297,7785.00,8153.87,8558.03,8560.74,8168.39,8300.09,False,True,True,8170.79,8190.82,False


In [68]:
# to check
df['take_profit'] = ((df['buy_sig'] == True) & (df['next_close_2'] > df['next_open_1']))
                     # & (df['high'] > df['prev_high_1']))
df[df['take_profit'] == True]

,datetime,symbol,open,high,low,close,volume,prev_low_1,prev_high_1,next_close_3,next_close_2,next_close_1,next_close_5,attack_buy_pattern,prev_is_abp,buy_sig,next_open_2,next_open_1,take_profit
179,2018-02-12 08:00:00,Binance:BTCUSDT,8063.82,8989.00,8053.00,8903.00,41987.850490,7726.53,8549.00,10000.09,9449.99,8539.90,11039.55,False,True,True,8535.17,8903.00,True
285,2018-05-29 08:00:00,Binance:BTCUSDT,7099.00,7540.00,7032.95,7461.29,39406.839520,7058.02,7437.00,7521.01,7485.01,7375.96,7714.26,False,True,True,7382.07,7467.98,True
316,2018-06-29 08:00:00,Binance:BTCUSDT,5851.01,6300.00,5780.00,6197.92,33911.287278,5827.00,6173.01,6615.29,6356.81,6390.07,6586.98,False,True,True,6391.08,6197.92,True
444,2018-11-04 08:00:00,Binance:BTCUSDT,6388.00,6525.00,6359.00,6485.85,10592.394943,6345.00,6439.97,6578.46,6519.11,6468.99,6419.99,False,True,True,6468.99,6485.85,True
565,2019-03-05 08:00:00,Binance:BTCUSDT,3716.10,3877.10,3703.55,3857.73,32962.536162,3670.69,3830.00,3864.89,3873.64,3861.84,3916.82,False,True,True,3861.84,3857.58,True
662,2019-06-10 08:00:00,Binance:BTCUSDT,7627.57,8020.00,7511.00,7982.75,36756.078468,7506.66,7935.00,8218.54,8127.64,7884.90,8808.70,False,True,True,7884.90,7981.00,True
781,2019-10-07 08:00:00,Binance:BTCUSDT,7855.30,8299.92,7762.00,8190.09,52202.072297,7785.00,8153.87,8558.03,8560.74,8168.39,8300.09,False,True,True,8170.79,8190.82,True
869,2020-01-03 08:00:00,Binance:BTCUSDT,6965.49,7405.00,6871.04,7344.96,68428.500451,6924.74,7212.50,7758.00,7358.75,7354.11,8055.98,False,True,True,7354.19,7345.00,True
902,2020-02-05 08:00:00,Binance:BTCUSDT,9197.02,9744.45,9177.22,9612.04,64870.415615,9093.01,9350.00,9895.05,9813.73,9772.00,9851.83,False,True,True,9772.00,9612.03,True
908,2020-02-11 08:00:00,Binance:BTCUSDT,9851.74,10323.59,9700.00,10223.08,62422.395224,9756.00,10188.00,10344.36,10229.63,10326.46,9917.27,False,True,True,10325.33,10223.08,True


In [69]:
len(df[df['take_profit'] == True])/len(df[df['buy_sig'] == True])

0.5555555555555556

In [52]:
df['profit_rate'] = (df['next_close_2'] - df['prev_high_1']) / df['prev_high_1']
df[df['take_profit'] == True]['profit_rate'].mean()

0.040313456276434484

In [53]:
df[df['prev_is_abp']==True]['profit_rate'].mean()

-0.04944443309516538